<a href="https://colab.research.google.com/github/Naima-Ashfaq12/DataScience/blob/main/Assignment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import io

# Upload dataset
uploaded = files.upload()

# Read CSV
df = pd.read_csv(io.BytesIO(uploaded['cleaned_student_dataset.csv']))

df.head()


Saving cleaned_student_dataset.csv to cleaned_student_dataset.csv


,first_name,last_name,gender,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,career_aspiration,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,geography_score
0,Paul,Casey,male,False,3,False,27,lawyer,73,81,93,97,63,80,87
1,Danielle,Sandoval,female,False,2,False,47,doctor,90,86,96,100,90,88,90
2,Tina,Andrews,female,False,9,True,13,government officer,81,97,95,96,65,77,94
3,Tara,Clark,female,False,5,False,3,artist,71,74,88,80,89,63,86
4,Anthony,Campos,male,False,5,False,10,unknown,84,77,65,65,80,74,76


In [2]:
# Create average score
df['average_score'] = df[['math_score', 'history_score', 'physics_score',
                          'chemistry_score', 'biology_score',
                          'english_score', 'geography_score']].mean(axis=1)

# Binary target: 1 = good performance, 0 = needs improvement
df['Performance'] = np.where(df['average_score'] >= 60, 1, 0)

df[['average_score', 'Performance']].head()


,average_score,Performance
0,82.000000,1
1,91.428571,1
2,86.428571,1
3,78.714286,1
4,74.428571,1


In [3]:
# Drop unnecessary columns
X = df.drop(['first_name', 'last_name', 'Performance', 'average_score'], axis=1)
y = df['Performance']

# One-hot encoding for categorical variables
X = pd.get_dummies(X, drop_first=True)

X.head()


,part_time_job,absence_days,extracurricular_activities,weekly_self_study_hours,math_score,history_score,physics_score,chemistry_score,biology_score,english_score,...,career_aspiration_game developer,career_aspiration_government officer,career_aspiration_lawyer,career_aspiration_real estate developer,career_aspiration_scientist,career_aspiration_software engineer,career_aspiration_stock investor,career_aspiration_teacher,career_aspiration_unknown,career_aspiration_writer
0,False,3,False,27,73,81,93,97,63,80,...,False,False,True,False,False,False,False,False,False,False
1,False,2,False,47,90,86,96,100,90,88,...,False,False,False,False,False,False,False,False,False,False
2,False,9,True,13,81,97,95,96,65,77,...,False,True,False,False,False,False,False,False,False,False
3,False,5,False,3,71,74,88,80,89,63,...,False,False,False,False,False,False,False,False,False,False
4,False,5,False,10,84,77,65,65,80,74,...,False,False,False,False,False,False,False,False,True,False


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize ANN
model = Sequential()

# Input + Hidden Layer 1
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden Layer 2
model.add(Dense(16, activation='relu'))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,473 (5.75 KB)

 Trainable params: 1,473 (5.75 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.8958 - loss: 0.3793 - val_accuracy: 0.9969 - val_loss: 0.1282
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0882 - val_accuracy: 0.9969 - val_loss: 0.0550
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0305 - val_accuracy: 0.9969 - val_loss: 0.0384
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0133 - val_accuracy: 0.9969 - val_loss: 0.0339
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0076 - val_accuracy: 0.9969 - val_loss: 0.0327
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.9969 - val_loss: 0.0326
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 0.9969 - val_loss: 0.0329
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9969 - val_loss

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predictions
y_pred_ann = (model.predict(X_test) > 0.5).astype(int)

# Metrics
accuracy_ann = accuracy_score(y_test, y_pred_ann)
precision_ann = precision_score(y_test, y_pred_ann)
recall_ann = recall_score(y_test, y_pred_ann)
f1_ann = f1_score(y_test, y_pred_ann)

print("ANN Performance:")
print("Accuracy:", round(accuracy_ann, 4))
print("Precision:", round(precision_ann, 4))
print("Recall:", round(recall_ann, 4))
print("F1 Score:", round(f1_ann, 4))


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
ANN Performance:
Accuracy: 0.9975
Precision: 0.9975
Recall: 1.0
F1 Score: 0.9987
